<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/llm_matrix_compuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By defining multiple keys (matrix_data_1, matrix_data_2, etc.) in your function schema, the LLM can return as many distinct matrices (or other data) as you need. Each matrix is an array of arrays, so you can parse them separately and do further processing in Python. Meanwhile, you still get a LaTeX expression to show the user in a nicely formatted way.

Creating and Using Your OpenAI API Key:

To do this head to the OpenAI API Dashboard and create/login to an account. (https://platform.openai.com/assistants).
After logging into an account, on the left side of the dashboard select ‘API Keys’.
Next, click ‘Create new secret key’ in the top right corner of the page, from here you can create a name for your key, the project it is used for (if you have multiple keys and projects), and decide the permissions.
After deciding the settings for your key, click ‘Create secret key’ in the bottom right of the menu.
This will show your given key, make sure to copy your key and save it in a place of your choosing for later. This will be the only time you can access it. Afterwords, select ‘Done.’   

You have to add at least 5 dollars to your account. In most case you probably use only less than dollar in the whole semester.

In [58]:
!pip install openai requests -q


In [30]:
import openai
import ast  # for safely evaluating Pythonic list representations (optional)
import numpy as np  # to make matrix multiplication easier (optional)
import json
from openai import OpenAI

client = OpenAI(
    #replace your openai API key. Make sure keep the key in a safe place,  colab and github do not save your jpynb file with the actual key.
    api_key="sk-pr**A"
)

In [31]:
functions = [
    {
        "name": "compute_matrices",
        "description": "Return a LaTeX expression plus two separate matrices.",
        "parameters": {
            "type": "object",
            "properties": {
                "latex_expression": {
                    "type": "string",
                    "description": "LaTeX expression showing the math."
                },
                "matrix_data_1": {
                    "type": "array",
                    "items": {
                        "type": "array",
                        "items": {"type": "number"}
                    },
                    "description": "First matrix as a 2D array."
                },
                "matrix_data_2": {
                    "type": "array",
                    "items": {
                        "type": "array",
                        "items": {"type": "number"}
                    },
                    "description": "Second matrix as a 2D array."
                }
            },
            "required": [
                "latex_expression",
                "matrix_data_1",
                "matrix_data_2"
            ]
        }
    }
]



In [34]:
def get_two_matrices_latex(matA, matB):
    """
    Ask the model to do some math with two matrices matA and matB,
    returning a LaTeX expression plus the resulting two matrices.
    """
    # Prompt example: let's say we want the LLM to add them, then also return the difference.
    user_prompt = (
        f"Given two matrices A={matA} and B={matB}, compute A+B and A-B. "
        "Return three things in JSON: 'latex_expression' showing the math, "
        "'matrix_data_1' for A+B, and 'matrix_data_2' for A-B. "
        "Use the function 'compute_matrices'."
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or gpt-4
        messages=[{"role": "user", "content": user_prompt}],
        functions=functions,
        function_call="auto"
    )

    msg = response.choices[0].message

    # If the LLM properly calls 'compute_matrices'
    if msg.function_call:
        arguments_str = msg.function_call.arguments
        data = json.loads(arguments_str)  # parse the JSON
        latex_expr = data["latex_expression"]
        matrix_1 = data["matrix_data_1"]  # e.g. result of A+B
        matrix_2 = data["matrix_data_2"]  # e.g. result of A-B
        return latex_expr, matrix_1, matrix_2
    else:
        return None, None, None


In [35]:

matA = [[1,2],[3,4]]
matB = [[5,6],[7,8]]
latex, sum_mat, diff_mat = get_two_matrices_latex(matA, matB)
print("LaTeX expression:", latex)
print("A+B:", sum_mat)
print("A-B:", diff_mat)


LaTeX expression: A + B = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix} + \begin{pmatrix} 5 & 6 \\ 7 & 8 \end{pmatrix} = \begin{pmatrix} 6 & 8 \\ 10 & 12 \end{pmatrix}, \quad A - B = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix} - \begin{pmatrix} 5 & 6 \\ 7 & 8 \end{pmatrix} = \begin{pmatrix} -4 & -4 \\ -4 & -4 \end{pmatrix}
A+B: [[6, 8], [10, 12]]
A-B: [[-4, -4], [-4, -4]]
